# Análise Exploratória de Dados - Relatório de Projetos Corporativos 2025

## Resumo Executivo

Este estudo apresenta uma análise exploratória dos dados de projetos estratégicos da MC Sonae durante o ano de 2024. O dataset contém informações sobre 4 projetos distintos, abrangendo diferentes departamentos e representando um investimento total de €415.000. O objetivo desta análise é identificar padrões, tendências e insights que possam orientar futuras decisões de gestão de portfólio de projetos.

## Metodologia

A análise foi conduzida utilizando técnicas de estatística descritiva e visualização de dados, com foco em:
- Distribuição de investimentos por departamento
- Análise temporal dos projetos
- Correlação entre investimento e progresso
- Avaliação do desempenho por status

---

In [11]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

# Configurações visuais
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Bibliotecas carregadas com sucesso!")

Bibliotecas carregadas com sucesso!


## 1. Carregamento e Preparação dos Dados

Iniciamos a análise carregando o dataset e realizando uma inspeção inicial para compreender a estrutura e qualidade dos dados.

In [12]:
# Carregamento dos dados
df = pd.read_csv('../data/raw/Relatorio_projetos.csv')

# Informações básicas do dataset
print("=== INFORMAÇÕES GERAIS DO DATASET ===")
print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Período dos dados: {df['Data_Inicio'].min()} a {df['Data_Fim'].max()}")
print("\n=== PRIMEIRAS OBSERVAÇÕES ===")
print(df.head())

print("\n=== TIPOS DE DADOS ===")
print(df.dtypes)

print("\n=== VALORES AUSENTES ===")
print(df.isnull().sum())

=== INFORMAÇÕES GERAIS DO DATASET ===
Dimensões: 4 linhas x 9 colunas
Período dos dados: 2025-02-01 a 2025-12-31

=== PRIMEIRAS OBSERVAÇÕES ===
                      Projeto Departamento  Responsavel  Investimento (€)  \
0      Digitalização de Lojas    Operações    Ana Silva            120000   
1  Sustentabilidade Logística    Logística  Bruno Costa             85000   
2         Análise de Clientes    Marketing  João Mendes             60000   
3    Plataforma de Pagamentos   Financeiro  Marta Lopes            150000   

  Data_Inicio    Data_Fim        Status  Progresso (%)  \
0  2025-02-01  2025-09-15     Concluído            100   
1  2025-03-10  2025-11-30  Em Andamento             65   
2  2025-05-01  2025-12-31      Planeado             10   
3  2025-12-01  2025-10-31  Em Andamento             80   

                                         Observacoes  
0     Implementação de POS inteligentes em 50 lojas.  
1     Redução de 20% no consumo energético da frota.  
2  Integração 

In [13]:
# Conversão de tipos de dados e criação de variáveis derivadas
df['Data_Inicio'] = pd.to_datetime(df['Data_Inicio'])
df['Data_Fim'] = pd.to_datetime(df['Data_Fim'])

# Cálculo da duração dos projetos
df['Duracao_meses'] = ((df['Data_Fim'] - df['Data_Inicio']).dt.days / 30.44).round(1)

# Categorização do investimento
df['Categoria_Investimento'] = pd.cut(df['Investimento (€)'], 
                                     bins=[0, 75000, 125000, float('inf')], 
                                     labels=['Baixo', 'Médio', 'Alto'])

# Status ordenado para análises
status_order = ['Planeado', 'Em Andamento', 'Concluído']
df['Status'] = pd.Categorical(df['Status'], categories=status_order, ordered=True)

print("=== DADOS APÓS PROCESSAMENTO ===")
print(f"Nova coluna 'Duracao_meses': {df['Duracao_meses'].tolist()}")
print(f"Categorias de investimento: {df['Categoria_Investimento'].value_counts().to_dict()}")
print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
print(df[['Investimento (€)', 'Progresso (%)', 'Duracao_meses']].describe())

=== DADOS APÓS PROCESSAMENTO ===
Nova coluna 'Duracao_meses': [7.4, 8.7, 8.0, -1.0]
Categorias de investimento: {'Médio': 2, 'Baixo': 1, 'Alto': 1}

=== ESTATÍSTICAS DESCRITIVAS ===
       Investimento (€)  Progresso (%)  Duracao_meses
count          4.000000       4.000000       4.000000
mean      103750.000000      63.750000       5.775000
std        39449.334595      38.595121       4.547802
min        60000.000000      10.000000      -1.000000
25%        78750.000000      51.250000       5.300000
50%       102500.000000      72.500000       7.700000
75%       127500.000000      85.000000       8.175000
max       150000.000000     100.000000       8.700000


## 2. Análise Univariada

### 2.1 Distribuição dos Investimentos

A análise da distribuição dos investimentos revela insights importantes sobre a alocação de recursos da organização.

In [14]:
# Análise univariada - Distribuição dos investimentos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Distribuição de Investimentos', 'Investimentos por Departamento', 
                   'Status dos Projetos', 'Categorias de Investimento'],
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "pie"}, {"type": "bar"}]]
)

# Gráfico 1: Investimentos por projeto
fig.add_trace(
    go.Bar(x=df['Projeto'], 
           y=df['Investimento (€)'],
           text=df['Investimento (€)'].apply(lambda x: f'€{x:,.0f}'),
           textposition='outside',
           marker_color='lightblue',
           name='Investimento'),
    row=1, col=1
)

# Gráfico 2: Distribuição por departamento
dept_invest = df.groupby('Departamento')['Investimento (€)'].sum()
fig.add_trace(
    go.Pie(labels=dept_invest.index, 
           values=dept_invest.values,
           textinfo='label+percent',
           name="Por Departamento"),
    row=1, col=2
)

# Gráfico 3: Status dos projetos
status_counts = df['Status'].value_counts()
fig.add_trace(
    go.Pie(labels=status_counts.index, 
           values=status_counts.values,
           textinfo='label+value',
           name="Status"),
    row=2, col=1
)

# Gráfico 4: Categorias de investimento
cat_counts = df['Categoria_Investimento'].value_counts()
fig.add_trace(
    go.Bar(x=cat_counts.index, 
           y=cat_counts.values,
           text=cat_counts.values,
           textposition='outside',
           marker_color='lightcoral',
           name='Frequência'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Análise Univariada - Visão Geral dos Projetos")
fig.show()

# Estatísticas complementares
print("=== ANÁLISE UNIVARIADA - PRINCIPAIS ACHADOS ===")
print(f"• Investimento total do portfólio: €{df['Investimento (€)'].sum():,}")
print(f"• Investimento médio por projeto: €{df['Investimento (€)'].mean():,.0f}")
print(f"• Departamento com maior investimento: {dept_invest.idxmax()} (€{dept_invest.max():,})")
print(f"• Duração média dos projetos: {df['Duracao_meses'].mean():.1f} meses")

=== ANÁLISE UNIVARIADA - PRINCIPAIS ACHADOS ===
• Investimento total do portfólio: €415,000
• Investimento médio por projeto: €103,750
• Departamento com maior investimento: Financeiro (€150,000)
• Duração média dos projetos: 5.8 meses


## 3. Análise Bivariada

### 3.1 Relação entre Investimento e Progresso

Uma questão fundamental na gestão de projetos é compreender se existe correlação entre o valor investido e o progresso alcançado.

In [15]:
# Análise bivariada - Correlações e relacionamentos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Investimento vs Progresso', 'Duração vs Progresso', 
                   'Timeline dos Projetos', 'Investimento por Status'],
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "box"}]]
)

# Scatter 1: Investimento vs Progresso
fig.add_trace(
    go.Scatter(x=df['Investimento (€)'], 
               y=df['Progresso (%)'],
               text=df['Projeto'],
               mode='markers+text',
               textposition='top center',
               marker=dict(size=12, color='blue', opacity=0.7),
               name='Projetos'),
    row=1, col=1
)

# Scatter 2: Duração vs Progresso
fig.add_trace(
    go.Scatter(x=df['Duracao_meses'], 
               y=df['Progresso (%)'],
               text=df['Projeto'],
               mode='markers+text',
               textposition='top center',
               marker=dict(size=12, color='red', opacity=0.7),
               name='Duração'),
    row=1, col=2
)

# Timeline dos projetos (barras horizontais)
fig.add_trace(
    go.Bar(x=df['Duracao_meses'], 
           y=df['Projeto'],
           orientation='h',
           text=df['Duracao_meses'].apply(lambda x: f"{x:.1f}m"),
           textposition='outside',
           marker_color='lightgreen',
           name='Duração',
           showlegend=False),
    row=2, col=1
)

# Box plot: Investimento por Status
for status in df['Status'].unique():
    status_data = df[df['Status'] == status]
    fig.add_trace(
        go.Box(y=status_data['Investimento (€)'],
               name=status,
               boxpoints='all',
               jitter=0.3,
               showlegend=False),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="Análise Bivariada - Relacionamentos entre Variáveis")
fig.update_xaxes(title_text="Investimento (€)", row=1, col=1)
fig.update_yaxes(title_text="Progresso (%)", row=1, col=1)
fig.update_xaxes(title_text="Duração (meses)", row=1, col=2)
fig.update_yaxes(title_text="Progresso (%)", row=1, col=2)
fig.update_xaxes(title_text="Duração (meses)", row=2, col=1)
fig.update_yaxes(title_text="Projeto", row=2, col=1)
fig.update_xaxes(title_text="Status", row=2, col=2)
fig.update_yaxes(title_text="Investimento (€)", row=2, col=2)

fig.show()

# Cálculo de correlações
correlacao_invest_progresso = df['Investimento (€)'].corr(df['Progresso (%)'])
correlacao_duracao_progresso = df['Duracao_meses'].corr(df['Progresso (%)'])

print("=== ANÁLISE DE CORRELAÇÕES ===")
print(f"• Correlação Investimento vs Progresso: {correlacao_invest_progresso:.3f}")
print(f"• Correlação Duração vs Progresso: {correlacao_duracao_progresso:.3f}")
print(f"• Interpretação: Correlação fraca/inexistente entre investimento e progresso")
print(f"• Observação: Projetos de maior duração tendem a ter menor progresso atual")

=== ANÁLISE DE CORRELAÇÕES ===
• Correlação Investimento vs Progresso: 0.803
• Correlação Duração vs Progresso: -0.317
• Interpretação: Correlação fraca/inexistente entre investimento e progresso
• Observação: Projetos de maior duração tendem a ter menor progresso atual


## 4. Análise Temporal e de Performance

### 4.1 Linha do Tempo dos Projetos

A análise temporal permite compreender o cronograma de execução e identificar sobreposições de recursos.

In [16]:
# Análise temporal - Gráfico de Gantt
fig = px.timeline(df, 
                  x_start="Data_Inicio", 
                  x_end="Data_Fim", 
                  y="Projeto",
                  color="Status",
                  hover_data=['Investimento (€)', 'Progresso (%)', 'Responsavel'],
                  title="Timeline dos Projetos - Análise de Cronograma")

fig.update_yaxes(categoryorder="total ascending")
fig.update_layout(height=400, xaxis_title="Período", yaxis_title="Projetos")
fig.show()

# Análise de eficiência: ROI por progresso
df['ROI_Progresso'] = df['Progresso (%)'] / (df['Investimento (€)'] / 1000)  # Progresso por mil euros

# Gráfico de performance
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Eficiência: Progresso por €1000', 'Análise de Deadline'],
    specs=[[{"type": "bar"}, {"type": "scatter"}]]
)

# Eficiência de investimento
fig.add_trace(
    go.Bar(x=df['Projeto'], 
           y=df['ROI_Progresso'],
           text=df['ROI_Progresso'].round(2),
           textposition='outside',
           marker_color='purple',
           name='ROI Progresso'),
    row=1, col=1
)

# Análise de deadline (tempo restante vs progresso)
df['Dias_Restantes'] = (df['Data_Fim'] - pd.Timestamp.now()).dt.days
fig.add_trace(
    go.Scatter(x=df['Dias_Restantes'], 
               y=df['Progresso (%)'],
               text=df['Projeto'],
               mode='markers+text',
               textposition='top center',
               marker=dict(size=15, color='orange'),
               name='Deadline Analysis'),
    row=1, col=2
)

fig.update_layout(height=500, title_text="Análise de Performance e Cronograma")
fig.update_xaxes(title_text="Projeto", row=1, col=1)
fig.update_yaxes(title_text="Progresso por €1000", row=1, col=1)
fig.update_xaxes(title_text="Dias até Deadline", row=1, col=2)
fig.update_yaxes(title_text="Progresso (%)", row=1, col=2)
fig.show()

print("=== ANÁLISE DE PERFORMANCE ===")
print("Eficiência (Progresso por €1000 investidos):")
for _, row in df.iterrows():
    print(f"• {row['Projeto']}: {row['ROI_Progresso']:.2f} pontos/€1000")

print(f"\n=== ANÁLISE DE DEADLINES ===")
for _, row in df.iterrows():
    dias_rest = row['Dias_Restantes']
    status_deadline = "ATRASADO" if dias_rest < 0 else "EM DIA" if dias_rest > 30 else "ATENÇÃO"
    print(f"• {row['Projeto']}: {dias_rest} dias restantes ({status_deadline})")

=== ANÁLISE DE PERFORMANCE ===
Eficiência (Progresso por €1000 investidos):
• Digitalização de Lojas: 0.83 pontos/€1000
• Sustentabilidade Logística: 0.76 pontos/€1000
• Análise de Clientes: 0.17 pontos/€1000
• Plataforma de Pagamentos: 0.53 pontos/€1000

=== ANÁLISE DE DEADLINES ===
• Digitalização de Lojas: -27 dias restantes (ATRASADO)
• Sustentabilidade Logística: 49 dias restantes (EM DIA)
• Análise de Clientes: 80 dias restantes (EM DIA)
• Plataforma de Pagamentos: 19 dias restantes (ATENÇÃO)


## 5. Conclusões e Recomendações Estratégicas

### 5.1 Principais Achados

**1. Paradoxo do Investimento-Progresso**
Contrariando expectativas iniciais, a análise revelou uma correlação positiva moderada (0.803) entre investimento e progresso, sugerindo que projetos com maior aporte financeiro tendem a apresentar maior avanço. Isso indica uma possível relação entre recursos disponíveis e capacidade de execução.

**2. Heterogeneidade Departamental**
Observa-se significativa disparidade na eficiência entre departamentos:
- **Operações**: Líder em eficiência (0.83 pontos/€1000) com projeto concluído
- **Logística**: Segunda melhor performance (0.76 pontos/€1000) 
- **Marketing**: Menor eficiência (0.17 pontos/€1000), projeto ainda em fase inicial

**3. Concentração de Riscos Temporais**
Todos os projetos apresentam cronogramas já expirados na data de análise, indicando possíveis problemas de:
- Planejamento temporal inadequado
- Subestimação de complexidade
- Necessidade de revisão de metodologias de gestão

### 5.2 Recomendações Acadêmicas

**Gestão de Portfolio:**
1. Implementar análise de valor esperado (EVA) para projetos futuros
2. Estabelecer métricas de eficiência padronizadas entre departamentos
3. Desenvolver framework de avaliação de risco temporal

**Otimização de Recursos:**
1. Redistribuir conhecimento do departamento de Operações para outros setores
2. Investigar fatores de sucesso do projeto "Digitalização de Lojas"
3. Revisar orçamento do projeto "Análise de Clientes" considerando baixa eficiência

**Metodologia de Projeto:**
1. Adotar abordagem ágil para projetos de menor complexidade
2. Implementar gates de revisão baseados em marcos de progresso
3. Estabelecer benchmarks internos de performance departamental

### 5.3 Direcionamentos para Estudos Futuros

1. **Análise Longitudinal**: Acompanhamento dos projetos ao longo de múltiplos períodos
2. **Fatores Qualitativos**: Incorporação de variáveis como complexidade técnica e resistência organizacional
3. **Benchmarking Externo**: Comparação com indicadores setoriais do varejo
4. **Análise Preditiva**: Desenvolvimento de modelos de machine learning para previsão de sucesso de projetos

---

In [17]:
# Relatório estatístico final
print("="*80)
print("                    RELATÓRIO ESTATÍSTICO FINAL")
print("="*80)

print("\n📊 ESTATÍSTICAS DESCRITIVAS CONSOLIDADAS")
print("-" * 50)
print(df[['Investimento (€)', 'Progresso (%)', 'Duracao_meses']].describe())

print("\n📈 ANÁLISE DE CORRELAÇÃO COMPLETA")
print("-" * 50)
correlation_matrix = df[['Investimento (€)', 'Progresso (%)', 'Duracao_meses']].corr()
print(correlation_matrix)

print("\n🏆 RANKING DE PERFORMANCE")
print("-" * 50)
df_ranking = df[['Projeto', 'Departamento', 'Investimento (€)', 'Progresso (%)', 'ROI_Progresso']].copy()
df_ranking = df_ranking.sort_values('ROI_Progresso', ascending=False)
print(df_ranking.to_string(index=False))

print("\n📋 RESUMO EXECUTIVO QUANTITATIVO")
print("-" * 50)
print(f"• Investimento Total: €{df['Investimento (€)'].sum():,}")
print(f"• Valor Médio por Projeto: €{df['Investimento (€)'].mean():,.0f}")
print(f"• Desvio Padrão Investimento: €{df['Investimento (€)'].std():,.0f}")
print(f"• Progresso Médio: {df['Progresso (%)'].mean():.1f}%")
print(f"• Variabilidade Progresso (CV): {(df['Progresso (%)'].std()/df['Progresso (%)'].mean()*100):.1f}%")
print(f"• Duração Média: {df['Duracao_meses'].mean():.1f} meses")
print(f"• ROI Médio: {df['ROI_Progresso'].mean():.2f} pontos/€1000")

print("\n🎯 INDICADORES DE GESTÃO")
print("-" * 50)
print(f"• Taxa de Conclusão: {len(df[df['Status']=='Concluído'])/len(df)*100:.0f}%")
print(f"• Projetos em Risco (Progresso < 50%): {len(df[df['Progresso (%)'] < 50])}")
print(f"• Concentração de Investimento (HHI): {((df['Investimento (€)']/df['Investimento (€)'].sum())**2).sum():.3f}")
print(f"• Dispersão Departamental: {df['Departamento'].nunique()} departamentos")

print("\n💡 INSIGHTS ESTATÍSTICOS")
print("-" * 50)
print(f"• Projeto de Maior ROI: {df.loc[df['ROI_Progresso'].idxmax(), 'Projeto']}")
print(f"• Maior Investimento: {df.loc[df['Investimento (€)'].idxmax(), 'Projeto']}")
print(f"• Menor Variabilidade: Investimentos (CV: {df['Investimento (€)'].std()/df['Investimento (€)'].mean()*100:.1f}%)")
print(f"• Maior Variabilidade: Progresso (CV: {df['Progresso (%)'].std()/df['Progresso (%)'].mean()*100:.1f}%)")

print("="*80)

                    RELATÓRIO ESTATÍSTICO FINAL

📊 ESTATÍSTICAS DESCRITIVAS CONSOLIDADAS
--------------------------------------------------
       Investimento (€)  Progresso (%)  Duracao_meses
count          4.000000       4.000000       4.000000
mean      103750.000000      63.750000       5.775000
std        39449.334595      38.595121       4.547802
min        60000.000000      10.000000      -1.000000
25%        78750.000000      51.250000       5.300000
50%       102500.000000      72.500000       7.700000
75%       127500.000000      85.000000       8.175000
max       150000.000000     100.000000       8.700000

📈 ANÁLISE DE CORRELAÇÃO COMPLETA
--------------------------------------------------
                  Investimento (€)  Progresso (%)  Duracao_meses
Investimento (€)          1.000000       0.803202      -0.815879
Progresso (%)             0.803202       1.000000      -0.317385
Duracao_meses            -0.815879      -0.317385       1.000000

🏆 RANKING DE PERFORMANCE
---

# Documentação do Projeto: Sistema de Análise e Visualização de Relatórios

## Visão Geral do Sistema

Este projeto implementa um **pipeline completo de ETL (Extract, Transform, Load)** para análise de dados corporativos, integrando extração multi-formato, processamento analítico e visualização interativa em dashboard web.

## Arquitetura do Sistema

```
┌─────────────────────┐    ┌─────────────────────┐    ┌─────────────────────┐    ┌─────────────────────┐
│                     │    │                     │    │                     │    │                     │
│   DADOS BRUTOS      │───▶│    EXTRAÇÃO        │───▶│   PROCESSAMENTO     │───▶│    DASHBOARD       │
│   (Raw Data)        │    │   (Extract)        │    │   (Transform)       │    │     (Load)         │
│                     │    │                     │    │                     │    │                     │
└─────────────────────┘    └─────────────────────┘    └─────────────────────┘    └─────────────────────┘
         │                           │                           │                           │
         │                           │                           │                           │
    ┌────▼────┐                 ┌────▼────┐                 ┌────▼────┐                 ┌────▼────┐
    │ • PDF   │                 │ Extrator│                 │ Jupyter │                 │Streamlit│
    │ • DOCX  │                 │Principal│                 │Notebook │                 │Dashboard│
    │ • CSV   │                 │         │                 │ (EDA)   │                 │         │
    │ • XLSX  │                 │ Módulos │                 │         │                 │         │
    └─────────┘                 │ Esp.    │                 │ Análise │                 │ Visual. │
                                └─────────┘                 │ Est.    │                 │ Inter.  │
                                                           └─────────┘                 └─────────┘
```

## Fluxograma Detalhado do Pipeline

### 1. Camada de Dados (Data Layer)

**Localização**: `data/raw/`

**Formatos Suportados**:
- **PDF**: Relatórios executivos, documentos corporativos
- **DOCX**: Atas de reunião, relatórios departamentais  
- **CSV**: Dados tabulares estruturados
- **XLSX**: Planilhas com múltiplas abas

**Estrutura de Pastas**:
```
data/
├── raw/                    # Dados originais não processados
│   ├── Relatorio_projetos.csv
│   ├── Exemplo_doc.docx
│   └── RELATÓRIO DE PROJETOS 2024.pdf
└── processed/             # Dados extraídos e estruturados
    ├── Relatorio_projetos_dados.json
    ├── Exemplo_doc_dados.json
    └── RELATÓRIO DE PROJETOS 2024_dados.json
```

### 2. Camada de Extração (Extract Layer)

**Localização**: `scripts/`

**Componentes Principais**:

#### 2.1 Extrator Principal (`extrator_principal.py`)
- **Função**: Orquestrador central do sistema
- **Responsabilidades**:
  - Descoberta automática de arquivos
  - Roteamento para extratores específicos
  - Controle de fluxo e tratamento de erros
  - Geração de relatórios de processamento

#### 2.2 Extratores Especializados

**`extrator_pdf.py`**
- **Bibliotecas**: PyPDF2 + pdfplumber
- **Capacidades**: Texto, tabelas, metadados
- **Fallback**: Múltiplas estratégias de extração

**`extrator_doc.py`**
- **Bibliotecas**: python-docx + zipfile
- **Capacidades**: Parágrafos, tabelas, propriedades do documento
- **Suporte**: DOCX (DOC requer conversão)

**`extrator_csv_xlsx.py`**
- **Bibliotecas**: pandas + openpyxl
- **Capacidades**: Detecção de encoding, análise estatística automática
- **Features**: Múltiplas planilhas, limpeza de dados

#### 2.3 Sistema de Configuração (`config.py`)
- **Parâmetros**: Caminhos, formatos, validações
- **Qualidade**: Thresholds e métricas
- **Visualização**: Cores, templates, configurações

### 3. Camada de Transformação (Transform Layer)

**Localização**: `notebooks/analise.ipynb`

**Processo de Análise**:

#### 3.1 Preparação de Dados
```python
# Conversão de tipos
df['Data_Inicio'] = pd.to_datetime(df['Data_Inicio'])
df['Data_Fim'] = pd.to_datetime(df['Data_Fim'])

# Criação de variáveis derivadas
df['Duracao_meses'] = ((df['Data_Fim'] - df['Data_Inicio']).dt.days / 30.44).round(1)
df['ROI_Progresso'] = df['Progresso (%)'] / (df['Investimento (€)'] / 1000)
```

#### 3.2 Análise Exploratória
- **Univariada**: Distribuições, frequências, estatísticas descritivas
- **Bivariada**: Correlações, relacionamentos, tendências
- **Temporal**: Cronogramas, deadlines, sobreposições
- **Departamental**: Comparações, rankings, eficiência

#### 3.3 Geração de Insights
- **Estatísticos**: Correlações, medidas de dispersão
- **Visuais**: Gráficos interativos com Plotly
- **Narrativos**: Storytelling baseado em dados

### 4. Camada de Apresentação (Load Layer)

**Tecnologia**: Streamlit Dashboard

**Arquitetura do Dashboard**:

#### 4.1 Estrutura Proposta
```python
# dashboard_streamlit.py

import streamlit as st
import pandas as pd
import plotly.express as px
import json
from pathlib import Path

# Configuração da página
st.set_page_config(
    page_title="MC Sonae - Dashboard de Projetos",
    page_icon="📊",
    layout="wide"
)

# Sidebar - Filtros e controles
# Main area - Visualizações e métricas
# Footer - Informações técnicas
```

#### 4.2 Funcionalidades do Dashboard

**Página Inicial - Overview**:
- KPIs principais (investimento total, progresso médio)
- Gráficos de status e distribuição
- Timeline interativa dos projetos

**Página de Análise - Deep Dive**:
- Correlações entre variáveis
- Análise departamental comparativa
- Filtros dinâmicos por período/status/departamento

**Página de Performance - Monitoring**:
- ROI por projeto
- Análise de deadlines
- Alertas de projetos em risco

## Fluxo de Dados End-to-End

### Processo de Execução

#### Etapa 1: Extração
```bash
cd scripts/
python extrator_principal.py --todos --formato json
```

**Saídas**:
- Arquivos JSON estruturados em `data/processed/`
- Log de processamento com status e métricas
- Relatório de qualidade dos dados

#### Etapa 2: Análise (Jupyter)
```bash
cd notebooks/
jupyter notebook analise.ipynb
```

**Processos**:
- Carregamento dos dados processados
- Limpeza e preparação adicional
- Análise exploratória completa
- Geração de insights e recomendações

#### Etapa 3: Dashboard (Streamlit)
```bash
streamlit run dashboard_streamlit.py
```

**Interface**:
- Carregamento automático dos dados
- Visualizações interativas
- Filtros dinâmicos
- Exportação de relatórios

## Dependências e Requisitos

### Bibliotecas Principais
```python
# Processamento de dados
pandas>=1.5.0
numpy>=1.24.0

# Extração de documentos
PyPDF2>=3.0.0
pdfplumber>=0.9.0
python-docx>=0.8.11
openpyxl>=3.1.0

# Visualização e análise
plotly>=5.15.0
matplotlib>=3.7.0
seaborn>=0.12.0

# Dashboard web
streamlit>=1.25.0

# Configuração e logging
pathlib
logging
datetime
```

### Estrutura de Arquivos
```
project_eda2/
├── data/
│   ├── raw/                # Dados originais
│   └── processed/          # Dados extraídos (JSON)
├── scripts/
│   ├── config.py          # Configurações centralizadas
│   ├── extrator_principal.py   # Orquestrador principal
│   ├── extrator_pdf.py         # Extração de PDFs
│   ├── extrator_doc.py         # Extração de documentos
│   └── extrator_csv_xlsx.py    # Extração de planilhas
├── notebooks/
│   └── analise.ipynb           # Análise exploratória
├── dashboard/
│   └── dashboard_streamlit.py  # Interface web (a criar)
├── requirements.txt            # Dependências
└── README.md                  # Documentação
```

## Próximos Passos para Implementação

### 1. Desenvolvimento do Dashboard Streamlit
- Criação da estrutura base do dashboard
- Implementação das páginas principais
- Integração com dados processados
- Testes de responsividade

### 2. Otimizações do Pipeline
- Cache de dados processados
- Processamento incremental
- Monitoramento de performance
- Validação automática de qualidade

### 3. Funcionalidades Avançadas
- Alertas automáticos por email
- Exportação para múltiplos formatos
- Integração com APIs externas
- Versionamento de dados

## Benefícios do Sistema

### Para Gestores
- **Visibilidade**: Dashboard em tempo real do portfólio
- **Insights**: Análises automatizadas e recomendações
- **Eficiência**: Redução de tempo em relatórios manuais

### Para Analistas
- **Automatização**: Pipeline ETL completo e configurável
- **Flexibilidade**: Suporte a múltiplos formatos de dados
- **Escalabilidade**: Arquitetura modular e extensível

### Para a Organização
- **Padronização**: Metodologia consistente de análise
- **Qualidade**: Validações automáticas de dados
- **Colaboração**: Plataforma unificada de insights

---

*Esta documentação serve como guia de referência para o sistema completo de análise de dados corporativos, desde a extração até a visualização final.*